In [43]:
import copy, math, matplotlib.pyplot as plt

In [44]:
genesis = {
    "AMM":{"r_1":100,"r_2":100,"s":100,"fee":0.00},
    "Trader":{"r_1":100,"r_2":100,"s":0},
    "LP":{"r_1":0,"r_2":0,"s":100}
    }

In [45]:
def swapToAsset2(state,inputs):
    agent = inputs[0]
    dA1 = inputs[1]
    feeFactor = (1-state["AMM"]["fee"])
    dA2 = state["AMM"]["r_2"]/(state["AMM"]["r_1"]+dA1*feeFactor)*dA1*feeFactor
    if dA1>0 and state[agent]["r_1"]-dA1 >= 0 :
        state["AMM"]["r_1"]+=dA1
        state[agent]["r_1"]-=dA1
        state["AMM"]["r_2"]-=dA2
        state[agent]["r_2"]+=dA2 
        
def swapToAsset1(state,inputs):
    agent = inputs[0]
    dA2 = inputs[1]
    feeFactor = (1-state["AMM"]["fee"])
    dA1 = state["AMM"]["r_1"]/(state["AMM"]["r_2"]+dA2*feeFactor)*dA2*feeFactor
    if dA2>0 and state[agent]["r_2"]-dA2 >= 0 :
        state["AMM"]["r_2"]+=dA2
        state[agent]["r_2"]-=dA2
        state["AMM"]["r_1"]-=dA1
        state[agent]["r_1"]+=dA1 

def addLiquidity(state,inputs):
    agent = inputs[0]
    R1=state["AMM"]["r_1"]
    R2=state["AMM"]["r_2"]
    S= state["AMM"]["s"]
    dA1=min(inputs[1],R1/R2*inputs[2])
    dA2=min(inputs[2],R2/R1*inputs[1])
    if (dA1 <= state[agent]["r_1"] and dA2 <= state[agent]["r_2"]) and (dA1 > 0 and dA2 > 0):
        state[agent]["r_1"]-=dA1
        state[agent]["r_2"]-=dA2
        state["AMM"]["r_1"]+=dA1
        state["AMM"]["r_2"]+=dA2
        dS = min(dA1/R1, dA2/R2) * S
        state["AMM"]["s"] += dS
        state[agent]["s"]+=dS
        
def removeLiquidity(state,inputs):
    dS = inputs[1]
    agent = inputs[0]
    if dS > 0 and state[agent]["s"]-dS>=0 and state["AMM"]["s"]-dS>=0:
        DR = (1-dS/state["AMM"]["s"])
        r_1=state["AMM"]["r_1"]
        r_2=state["AMM"]["r_2"]
        state[agent]["s"]-=dS
        state["AMM"]["r_1"]=r_1*DR
        state["AMM"]["r_2"]=r_2*DR
        state[agent]["r_1"]+=r_1-state["AMM"]["r_1"]
        state[agent]["r_2"]+=r_2-state["AMM"]["r_2"]
        state["AMM"]["s"]-=dS
        

def nice_print(self):
    if type(self)==float:
        return  round(self,3)
    if type(self)==int:
        return  self
    if type(self)==list:
        return  list(map(lambda l: print(l),self))
    if type(self)==dict:
        return  dict(map(lambda kv: (kv[0], print(kv[1])),self.items()))
    
def check_genesis_block(genesis, actionList):
    for item in actionList:
        if item[1][1] < 0:
            print(f"Error: a state block is not valid | {item}")
            return False
    return True
    
def evolve(state, actionStack):
    if not check_genesis_block(genesis, actionStack):
        return False
    history = [copy.deepcopy(state)]
    for action in actionStack:
        action[0](state,action[1])
        history.append(copy.deepcopy(state))
    return history 

def update_pool_fees(state, fee):
    state["AMM"]["fee"] = fee
    
def pctToAmount(pct):
    if pct > 1:
        return pct/100
    if pct < 1:
        return pct

def swapToAsset2_pct(state,inputs):
    swapToAsset2(state, [inputs[0], pctToAmount(inputs[1])(state[inputs[0]]["r_1"])])
    
def fiat_print_holdings(state, agent, token_1, token_2):
    holdings = state[agent][token_1]*token_1 + state[agent][token_2]*token_2
    print(f"{agent} holds {holdings}: {state[agent][token_1]} {token_1} and {state[agent][token_2]} {token_2}")
    
def invariant(state):
    return state["AMM"]["r_1"] * state["AMM"]["r_2"]

def asset1(state):
    return state["AMM"]["r_1"]+state["Trader"]["r_1"]+state["LP"]["r_1"]

def asset2(state):
    return state["AMM"]["r_2"]+state["Trader"]["r_2"]+state["LP"]["r_2"]

In [46]:
actionList = [
        [ swapToAsset2 , [  "Trader" , 50 ]],
        [ swapToAsset1 , [  "Trader" , 25 ]],
        [ swapToAsset2 , [  "Trader" , -50 ]],
]

In [47]:
evolve(genesis,actionList)
genesis

Error: a state block is not valid | [<function swapToAsset2 at 0x109ab9480>, ['Trader', -50]]


{'AMM': {'r_1': 100, 'r_2': 100, 's': 100, 'fee': 0.0},
 'Trader': {'r_1': 100, 'r_2': 100, 's': 0},
 'LP': {'r_1': 0, 'r_2': 0, 's': 100}}

In [48]:
genesis = {
    "AMM":{"r_1":100,"r_2":100,"s":100,"fee":0.0},
    "Trader":{"r_1":100,"r_2":100,"s":0},
    "LP":{"r_1":0,"r_2":0,"s":100}
    }

state = copy.deepcopy(genesis)

swapToAsset2(state,["Trader",13])

invariant(genesis)==invariant(state)

# set nonzero fees for the pool
update_pool_fees(state, 0.01)
# do a swap
swapToAsset1(state,["Trader",20])
# check the invariant with message
invariant(genesis)==invariant(state)

False

In [49]:
# testing token conservation individually
genesis = {
    "AMM":{"r_1":100,"r_2":100,"s":100,"fee":0.0},
    "Trader":{"r_1":100,"r_2":100,"s":0},
    "LP":{"r_1":0,"r_2":0,"s":100}
    }

state = copy.deepcopy(genesis)

swapToAsset1(state,["Trader",13])

asset1(genesis)==asset1(state) and asset2(genesis)==asset2(state)

True

In [50]:
# testing the failing test with manipulated state
genesis = {
    "AMM":{"r_1":100,"r_2":100,"s":100,"fee":0.0},
    "Trader":{"r_1":100,"r_2":100,"s":0},
    "LP":{"r_1":0,"r_2":0,"s":100}
    }

state = {
    "AMM":{"r_1":100,"r_2":100,"s":100,"fee":0.0},
    "Trader":{"r_1":0,"r_2":100,"s":0},
    "LP":{"r_1":0,"r_2":0,"s":100}
    }
    
asset1(genesis)==asset1(state) and asset2(genesis)==asset2(state)

False

In [51]:
# If you are doing the tests right, you usually produce far more source code for the tests than for the twin you are testing. It can easily be the case that the tests are twice as big as the system to be tested; even tests ten times bigger than the system of interest are nothing to worry about. Do not be greedy with your time when testing your system and try to think about as many test scenarios as possible! 

# We need to think about how the system should behave in more complex scenarios and which additional properties we can derive from that. One often important thing is reversibility: as long as the invariant is preserved, there should be an inverse action to every state transition.

In [52]:
# We make sure that if we make two swaps of the same amount in opposite direction, we are back where we started. 
genesis = {
    "AMM":{"r_1":100,"r_2":100,"s":100,"fee":0.0},
    "Trader":{"r_1":100,"r_2":100,"s":0},
    "LP":{"r_1":0,"r_2":0,"s":100}
    }

state = copy.deepcopy(genesis)

swapToAsset1(state,["Trader",13])

gained = state["Trader"]["r_1"] - genesis["Trader"]["r_1"]

swapToAsset2(state,["Trader",gained])

genesis == state

True

In [53]:
# Write a similar test for the liquidity systems that verifies that a genesis state is reproduced after inserting liquidity and removing it again.
genesis = {
    "AMM":{"r_1":100,"r_2":100,"s":100,"fee":0.0},
    "Trader":{"r_1":100,"r_2":100,"s":0},
    "LP":{"r_1":0,"r_2":0,"s":100}
    }

state = copy.deepcopy(genesis)

addLiquidity(state,["Trader",10,10])
# testing the invariant
invariant(genesis)==invariant(state)

removeLiquidity(state,["Trader",10])
invariant(genesis)==invariant(state)

True

In [54]:
# A longer simulation that implements multiple swap actions and plots the corresponding data. 

genesis = {
    "AMM":{"r_1":99,"r_2":1,"s":math.sqrt(99),"fee":0.0},
    "Trader":{"r_1":1,"r_2":99,"s":0},
    "Liquidator":{"r_1":200,"r_2":200,"s":100}
    }

actionList= [ [swapToAsset1,["Trader",1]] ] * 99

history = evolve(genesis, actionList)

AMM_r1=[]
Trader_r1=[]
AMM_r2=[]
Trader_r2=[]
for s in history:
    AMM_r1.append(s["AMM"]["r_1"])
    Trader_r1.append(s["Trader"]["r_1"])
    AMM_r2.append(s["AMM"]["r_2"])
    Trader_r2.append(s["Trader"]["r_2"])   


plt.figure(figsize=(10,4)) 
    
plt.subplot(1, 2, 1) 
plt.plot(AMM_r1,AMM_r2)
plt.title('AMM')
plt.xlabel("Asset 1")
plt.ylabel("Asset 2")

plt.subplot(1, 2, 2)
plt.plot(Trader_r1,Trader_r2)
plt.title('Trader')
plt.xlabel("Asset 1")
plt.ylabel("Asset 2")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/zakhar/Desktop/python/token eng/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
  File "/var/folders/6s/xfrpp0xd2wlfbjl7gn_y82qw0000gn/T/ipykernel_58609/3597744766.py", line 24, in <module>
    plt.figure(figsize=(10,4))
  File "/Users/zakhar/Desktop/python/token eng/.venv/lib/python3.10/site-packages/matplotlib/_api/deprecation.py", line 454, in wrapper
  File "/Users/zakhar/Desktop/python/token eng/.venv/lib/python3.10/site-packages/matplotlib/pyplot.py", line 783, in figure
  File "/Users/zakhar/Desktop/python/token eng/.venv/lib/python3.10/site-packages/matplotlib/pyplot.py", line 358, in new_figure_manager
  File "/Users/zakhar/Desktop/python/token eng/.venv/lib/python3.10/site-packages/matplotlib/pyplot.py", line 336, in _warn_if_gui_out_of_main_thread
  File "/Users/zakhar/Desktop/python/token eng/.venv/lib/python3.10/site-packages/matplotlib/pyplot.py", line 207, in _get_backend_mod
 